# Creamos la base de datos inicial con las compañías

Cogeremos las compañías listadas en el London Stock Exchange (LSE) y las que cotizan en Eurostoxx, porque son las que más market cap tienen y por ende más probabilidad de encontrarnos con el total de las compañías europeas. Las cotizadas en el LSE son solo las pertenecientes a la union europea y que tienen suficiente liquidez, regulado por EQS!!. 

links: 
LSE = https://www.londonstockexchange.com/securities-trading/equity-trading/uk-and-european-securities
EUROSTOXX = https://live.euronext.com/en/products/equities/list

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Prueba lectura html con índices eurostoxx y FTSE100

eurostoxx = pd.read_html('https://en.wikipedia.org/wiki/EURO_STOXX_50')[3]

euro50 = eurostoxx.drop(columns = [ 'Main listing', 'Corporate form', 'Founded'])

ftsie = pd.read_html('https://en.wikipedia.org/wiki/FTSE_100_Index')[4]
ftse100 = ftsie.rename(columns = {'Company': 'Name', 'FTSE Industry Classification Benchmark sector[14]' : 'Industry'})

# Creación de lista de tickers 

tickers_test = euro50.Ticker.to_list() + ftse100.Ticker.to_list()

#Prueba para buscar información sobre las compañías a tratar en el proyecto ^^

In [ ]:
# Recogemos los datos de las compañías del Euronext
path = '/Users/ruben/Documents/MASTER/TFM/data/'
euronext = pd.read_csv(path+'Euronext_Equities_2023-04-08.csv', sep = ';')
euronext.drop(index = [0,1,2], inplace = True)
# Limpiamos los datos para quedarnos con los que nos interesan
euronext_data = euronext.drop(columns = ['Open Price',
       'High Price', 'low Price', 'last Price', 'last Trade MIC Time',
       'Time Zone', 'Volume', 'Turnover', 'Closing Price', 'Closing Price DateTime'])
euronext_data.rename(columns={'Symbol': 'Ticker'}, inplace=True)
euronext_data

In [ ]:
# Guardar como CSV
#euronext_data_test = euronext.drop(columns = ['ISIN', 'Symbol', 'Market', 'Currency', 'Open Price',
       #'High Price', 'low Price', 'last Price', 'last Trade MIC Time',
       #'Time Zone', 'Volume', 'Turnover'])
#euronext_data_test.to_csv('/Users/ruben/Documents/MASTER/TFM/ESG.csv', index = False)

In [ ]:
# Test Companies EURONEXT
euronext_data[euronext_data['Ticker']=='KNEBV']

In [ ]:
# Companies listed on LSE different than those in Euronext

lse_companies = pd.read_excel(path+'list_of_eqs_securities_150.xls', header = 3).drop(0).drop(columns = 'Unnamed: 0')
intersec = list(set(euronext_data['ISIN']) & set(lse_companies['ISIN']))
lse_companies_2 = lse_companies.set_index('ISIN').drop(intersec).reset_index()

lse_companies_3 = lse_companies_2.drop(columns = ['Segment', 'Sector', 'Sedol', 'Security Type',
       'MiFIR Identifier', 'Currency', 'Trading Parameter Code',
       'Price Tick Table ID', 'Country of Register', 'Short Name',
       'Long Name', 'Professional Investor Only', 'EMS', 'Max Spread Floor',
       'Max Spread Perc.', 'ADT / PTS (EU)', 'ADT / PTS (UK)',
       'Issuer Version Start Date'])
lse_companies_3.rename(columns = {'Mnemonic': 'Ticker', 'Issuer Name': 'Name'}, inplace = True)
not_final = pd.concat([lse_companies_3, euronext_data], axis = 0)
not_final

In [ ]:
# Test companies LSE
lse_companies_2[lse_companies_2['ISIN']=='FI0009013403']

In [ ]:
# ESGBook companies in LSE

intersec_2 = list(set(df_e['company']) & set(lse_companies['Issuer Name']))
intersec_2

In [ ]:
# Buscamos los RICs de las compañías en Refinitiv

import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

In [ ]:
# retrieve RIC from ISIN
ISIN = not_final['ISIN'].to_list()

DATA, err = ek.get_data(ISIN, ['TR.RIC'])
 
# convert RICs to list to be used
ric_lists = DATA['RIC'].to_list()
# get data
#df, err = ek.get_data(ric_lists,
                      #['CF_NAME','CF_BID', 'CF_ASK', 'BIDSIZE', 'ASKSIZE', 'CF_EXCHNG'])
#display(df)

In [ ]:
# Eliminamos duplicados
RIC = list(dict.fromkeys(ric_lists))
len(RIC)

In [ ]:
# Eliminamos duplicados en el dataframe test
test = not_final.drop_duplicates(subset = 'ISIN')
test['RICS'] = RIC
len(test)
#vemos que el total de compañías entre las dos bolsas es de 3066, sin contar los duplicados por las acciones de doble clase

In [ ]:
#-localizamos los sectores desde refinitiv
sector, err = ek.get_data(ric_lists, ['TR.TRBCIndustry'])

In [ ]:
test['sector'] = sector['TRBC Industry Name']
test.to_csv(path+'test.csv', index = False)

In [ ]:
# Sacamos los sectores de las compañías para eliminar las que no sean de los sectores que nos interesan
names_sector = sector.groupby('TRBC Industry Name').count()
names_sector

In [ ]:
# Eliminamos de la lista los fondos de inversión 
nills = test[test['sector']== '']
to_del = nills['Name'][0:191]
test_2 = test.set_index('Name').drop(to_del).reset_index()
test_2.to_csv(path+'test.csv', index = False) # CSV con los datos de las compañías para empezar a eliminar setores específicos


In [3]:
test_2 = pd.read_csv('/Users/ruben/Documents/MASTER/TFM/data/test_2.csv')
test_2

,Name,ISIN,Ticker,Market,Currency,RICS,sector,sector_T
0,VOESTALPINE AG,AT0000937503,0MKX,NaN,NaN,VOES.VI,Iron & Steel,Iron & Steel
1,VIENNA INSURANCE GROUP AG WIENER VE,AT0000908504,0MZX,NaN,NaN,VIGR.VI,Multiline Insurance & Brokers,Multiline Insurance & Brokers
2,OESTERREICHISCHE POST AG,AT0000APOST4,0NTM,NaN,NaN,POST.VI,"Courier, Postal, Air Freight & Land-based Logi...","Courier, Postal, Air Freight & Land-based Logi..."
3,MAYR MELNHOF KARTON AG,AT0000938204,0NZ7,NaN,NaN,MMKV.VI,Paper Packaging,Paper Packaging
4,SCHOELLER BLECKMANN OILFIELD EQUIPM,AT0000946652,0O05,NaN,NaN,SBOE.VI,Oil Related Services and Equipment,Oil Related Services and Equipment
...,...,...,...,...,...,...,...,...
2870,ZENITH ENERGY,CA98936C1068,ZENA,Euronext Growth Oslo,NOK,ZENZ.L,Integrated Telecommunications Services,Oil & Gas Exploration and Production
2871,ZENOBE GRAMME CERT,BE0003809267,ZEN,Euronext Brussels,EUR,ZENG.BR,Integrated Telecommunications Services,"Real Estate Rental, Development & Operations"
2872,ZIGNAGO VETRO,IT0004171440,ZV,Euronext Milan,EUR,ZV.MI,Integrated Telecommunications Services,Non-Paper Containers & Packaging
2873,ZUCCHI,IT0005395071,ZUC,Euronext Milan,EUR,ZUCI.MI,Wireless Telecommunications Services,Apparel & Accessories


# Primer Filtro de los sectores a eliminar

Vamos a decidir cuales eliminamos de la lista anterior, bajo una serie de criterios basados en el impacto medioambiental del sector, como el sector petrolero, el del carbon. También es discutible eliminar los sectores relacionados con el gambling (apuestas) y el tabaco. Por último, también se puede eliminar el sector de la minería, ya que es un sector muy contaminante y que no tiene un impacto positivo en la sociedad.

Investigar y determinar de manera rígida el primer filtro de los sectores a eliminar.

Parece que agencias como S&P incluyen compañias que explotan petroleo en su índice, eliminando solo los que "extraen" el producto, pero no las refinadoras ni las que se benefician de su venta, pero en el caso del carbón si que las elimina a rodas en todas las secciones de la indústria. Por eso encontramos a Exxon mobile en su índice, pero no a Peabody Energy, que es una de las mayores productoras de carbón del mundo. Podemos eliminar: 

    * Petróleo 
    * Tabaco
    * Gambling
    * Weapons
    * Coal
    * Military
    * No ESG data Companies

También, excluimos compañías que esten blaclisted por la ONU, como las que tienen que ver con el terrorismo, el narcotráfico, etc. Podemos usar sustainalytics global standard screening para eso ya que incluye otras agencias interesantes. 

Vamos a eliminar tambien las compañías del sector *"Bitcoin and Cryptocurrency"*, primero por el impacto de la minería en el consumo energético global y segundo por la falta de transparencia y falta de datos ESG sobre las compañías en cuestión. 

Las compañías de la lista son todas Mid y Large Cap.

In [ ]:
#Actualizamos la base de datos de compañias y sectores (test_2.csv) que estaba erronea

sector, err = ek.get_data(test_2['RICS'].to_list(), ['TR.TRBCIndustry'])
probs = test_2[sector['TRBC Industry Name']!= test_2['sector']]

test_2['sector_T'] = sector['TRBC Industry Name'] # Corregimos poniendo la columna con los sectores correctos
test_2[test_2['RICS']=='VERN.EUA']

test_2.to_csv(path+'test_2.csv', index = False)

In [ ]:
# Buscamos los RICs de las compañías en Refinitiv

import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

In [ ]:
import pandas as pd 
import numpy as np 

path = '/Users/ruben/Documents/MASTER/TFM/data/'
test_2 = pd.read_csv(path + 'test_2.csv')

In [ ]:

sector_list = test_2['sector_T'].sort_values().unique().tolist() # Lista de sectores
sector_list # Para seleccionar los sectores que no nos interesan

In [ ]:
# Probamos seleccionar por sector

selected_sector = test_2.groupby('sector_T')
selected_sector.get_group('Aerospace & Defense')

In [ ]:
mask = test_2['ISIN'].str.contains('ES', case=False, na=False) # Podríamos Coger y hacer uno para Españolas También...  
test_2[mask]

In [ ]:
# Num de Compañías por sector

test_2.groupby('sector_T').count().sort_values('Name', ascending= False) 

In [ ]:
# Companies and Sectors out -> Primer Screening: 156 Compañías fuera

comp_out = ['DE0007030009','US75513E1010', 'SE0000112385', 'FR0000121329', 'FR0006174496']
sect_out = ['Aerospace & Defense', 
            'Blockchain & Cryptocurrency',
            'Casinos & Gaming',
            'Coal',
            'Integrated Oil & Gas',
            'Oil & Gas Drilling',
            'Oil & Gas Exploration and Production',
            'Oil & Gas Refining and Marketing',
            'Oil Related Services and Equipment',
            'Tobacco']
white_list = ['US36467X2062']

In [ ]:
test_3 = test_2
test_3 = test_3.drop(comp_out).reset_index()
test_3 = test_3.set_index('sector_T', inplace = False).drop(sect_out).reset_index()
test_3


In [ ]:
test_3.to_csv(path+'test_3.csv', index = False)

In [4]:
test_3 = pd.read_csv('/Users/ruben/Documents/MASTER/TFM/data/test_3.csv')
test_3

,sector_T,ISIN,Name,Ticker,Market,Currency,RICS,sector
0,Iron & Steel,AT0000937503,VOESTALPINE AG,0MKX,NaN,NaN,VOES.VI,Iron & Steel
1,Multiline Insurance & Brokers,AT0000908504,VIENNA INSURANCE GROUP AG WIENER VE,0MZX,NaN,NaN,VIGR.VI,Multiline Insurance & Brokers
2,"Courier, Postal, Air Freight & Land-based Logi...",AT0000APOST4,OESTERREICHISCHE POST AG,0NTM,NaN,NaN,POST.VI,"Courier, Postal, Air Freight & Land-based Logi..."
3,Paper Packaging,AT0000938204,MAYR MELNHOF KARTON AG,0NZ7,NaN,NaN,MMKV.VI,Paper Packaging
4,Semiconductors,AT0000969985,AT & S AUSTRIA TECHNOLOGIE & SYSTEM,0O5C,NaN,NaN,ATSV.VI,Semiconductors
...,...,...,...,...,...,...,...,...
2714,Specialty Mining & Metals,BMG9887P1068,ZCI LIMITED,CV,Euronext Paris,EUR,ZCIJ.J^G18,Multiline Insurance & Brokers
2715,"Real Estate Rental, Development & Operations",BE0003809267,ZENOBE GRAMME CERT,ZEN,Euronext Brussels,EUR,ZENG.BR,Integrated Telecommunications Services
2716,Non-Paper Containers & Packaging,IT0004171440,ZIGNAGO VETRO,ZV,Euronext Milan,EUR,ZV.MI,Integrated Telecommunications Services
2717,Apparel & Accessories,IT0005395071,ZUCCHI,ZUC,Euronext Milan,EUR,ZUCI.MI,Wireless Telecommunications Services


In [7]:
(2875-2719)/2875 # 5.4% de compañías fuera

0.05426086956521739

# Empezamos a recoger los scores de ESG de las compañías 

Cogeremos environmental, social and governance por diferentes fuentes, para empezar a seleccionar los mejor posicionados por score. A partir de ahí, crearemos un score completo a partir de los diferentes scores único para usar. 

In [ ]:
# Buscamos los RICs de las compañías en Refinitiv

import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

In [ ]:
# Sustainability by Yahoofinance reports (Sustainalytics) -> No funciona

import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np

path = '/Users/ruben/Documents/MASTER/TFM/data/'
test_3 = pd.read_csv(path + 'test_3.csv')

In [ ]:
# ESG Data de Refinitiv

comp_esg = test_3['ISIN'].to_list()
esg_ref, err = ek.get_data(comp_esg, ["TR.EnvironmentPillarScore","TR.SocialPillarScore","TR.GovernancePillarScore", 'TR.TRESGScore'])

In [ ]:
noesg_data_ref = esg_ref[esg_ref['Environmental Pillar Score'].isna()==True]['Instrument']
noesg_data_ref # Lista de compañías sin datos ESG en Refinitiv -> 1516 compañías
# Cuando tengamos los demás scores, comparamos compañías y eliminamos las que no tengan datos en los 3

In [ ]:
import requests
import time

url = "https://query2.finance.yahoo.com/v1/finance/search"
ticker_fin_1=[]
error = []
retry_count = 5

for i in range(10):
    try:
        for i in test_3['Ticker'][len(ticker_fin_1):]: 
            params = {'q': i, 'quotesCount': 1, 'newsCount': 0}
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
            response = requests.get(url, params=params, headers=headers)
            data = response.json()
            if data['quotes']!=[]:
                symbol = data['quotes'][0]['symbol']
                ticker_fin_1.append(symbol)
            else:
                error.append(i)
                ticker_fin_1.append('')
    except IndexError:
        print(f"An error occurred")
        if retry_count > 0:
            print(f"Retrying in 5 seconds... ({retry_count} attempts remaining)")
            time.sleep(5)
            retry_count -= 1
            continue
        else:
            print("Maximum retry attempts reached at Ticker number:", len(ticker_fin_1))
        break

ticker_fin_1

In [ ]:
test_3['Tickers_YF'] = ticker_fin_1
test_3

In [ ]:
no_ticker = test_3[test_3['Tickers_YF']=='']
y_ticker = test_3[test_3['Tickers_YF']!='']
y_ticker

In [ ]:
# yfinance esg scores
tickers = Ticker(y_ticker['Tickers_YF'].to_list())
esg_data_sust = tickers.esg_scores

In [ ]:
keys = pd.DataFrame(esg_data_sust.keys(), columns = ['Tickers_YF'])
keys_1 = keys['Tickers_YF'].to_list()
len(keys_1)

In [ ]:
y_or_n = list(set(keys_1) & set(y_ticker['Tickers_YF']))
y_ticker[y_ticker.isin({'Tickers_YF' : y_or_n})==False]

In [ ]:
scores = 'totalEsg', 'environmentScore', 'socialScore', 'governanceScore'
esg_scores_sust = pd.DataFrame(columns = scores, index= esg_data_sust.keys())
for v in esg_scores_sust.index:
    for i in scores:
        if v in esg_data_sust and i in esg_data_sust[v]:
            esg_scores_sust.loc[v, i] = esg_data_sust[v][i]
        else:
            print(f"Invalid key pair: ({v}, {i})")
esg_scores_sust


In [ ]:
esg_yf = y_ticker.merge(esg_scores_sust, how = 'left', left_on = 'Tickers_YF', right_on = esg_scores_sust.index)

In [ ]:
no_data = list(set(esg_ref.isna()) & set(esg_scores_sust.isna()))
no_data

In [ ]:
test_4 = test_3.set_index('ISIN').merge(esg_ref, how = 'left', left_index = True, right_index = True).reset_index()
test_4

In [ ]:
test_4 = test_4.merge(esg_scores_sust, how = 'left', left_index = True, right_index = True).reset_index()
test_4

In [ ]:
import math

test_5 = test_4
for i in test_5.index:
    if pd.isna(test_5['Environmental Pillar Score'].loc[i]) & math.isnan(test_5['environmentScore'].loc[i]):
        test_5.drop(i, inplace = True)
len(test_5)

In [ ]:
test_5.to_csv(path + 'test_5.csv')

In [13]:
test_5 = pd.read_csv('/Users/ruben/Documents/MASTER/TFM/data/test_5.csv')
test_5.totalEsg.isna().sum()

951

In [10]:
test_5

,ISIN,Tickers_YF,sector_T,Name,Ticker,Market,Currency,RICS,sector,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,ESG Score,totalEsg,environmentScore,socialScore,governanceScore
0,NL0012768675,NaN,"Medical Equipment, Supplies & Distribution",IGEA PHARMA NV,0AAX,NaN,NaN,IGPH.S,"Medical Equipment, Supplies & Distribution",0.000000,5.723266,36.310433,15.753078,NaN,NaN,NaN,NaN
1,DE0007037145,NaN,Multiline Utilities,RWE AG,0FUZ,NaN,NaN,RWEG_p.DE^G19,Multiline Utilities,79.801974,61.247367,71.522043,71.563817,NaN,NaN,NaN,NaN
2,DE0005501357,NaN,Consumer Publishing,AXEL SPRINGER SE,0NV2,NaN,NaN,SPRGn.H^B21,Consumer Publishing,21.013336,59.248939,40.482577,47.357783,NaN,NaN,NaN,NaN
3,DE000A12B8Z4,NaN,Real Estate Services,TLG IMMOBILIEN AG,0R3N,NaN,NaN,TLGG.H,Real Estate Services,4.289065,47.073310,29.460881,27.843789,NaN,NaN,NaN,NaN
4,ES0105200002,NaN,Construction & Engineering,ABENGOA SA,0QD0,NaN,NaN,ABGek.MC^I22,Construction & Engineering,78.381723,80.167024,45.274123,70.269056,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,NO0010708910,ZAL.DE,Employment Services,ZALARIS,ZAL,Oslo Børs,NOK,ZAL.OL,"Medical Equipment, Supplies & Distribution",20.177210,66.352726,26.111111,41.984059,13.49,2.31,6.09,5.11
1299,IT0004171440,ZVRA,Non-Paper Containers & Packaging,ZIGNAGO VETRO,ZV,Euronext Milan,EUR,ZV.MI,Integrated Telecommunications Services,70.491275,66.412219,73.324183,69.526507,NaN,NaN,NaN,NaN
1300,NO0010816093,ZWS,Specialty Chemicals,ELKEM,ELK,Oslo Børs,NOK,ELK.OL,Paper Products,75.827626,69.718169,83.424908,75.210100,NaN,NaN,NaN,NaN
1301,IT0005366601,^DJI,Industrial Machinery & Equipment,ANTARES VISION,AV,Euronext Milan,EUR,ANV.MI,Biotechnology & Medical Research,37.144539,39.913842,38.191908,38.476947,NaN,NaN,NaN,NaN


In [15]:
(1303 - 951)/(2719)


0.12945936005884517

# Seguimos recogiendo Scores de ESG

En la sección pasada, recogimos los datos de Refinitiv y de Sustainalytics (mediante Yahoo Finance), y eliminamos todas las de las que no tuvieramos datos en ninguna de las dos agencias. Lo ideal sería encontrar otra agencia de puntuación ESG, para tener una tercera fuente de datos, y volver a filtrar para las 3 que como mínimo tengamos 2 fuentes de información por compañía. 

Buenas opciones son MSCI, S&P, ESGBook puede ofrecer compañías interesantes que probablemte tengamos dentro de la base de datos también.

Los datos hasta ahora están guardados en *test_5*


In [16]:
test_7  = pd.read_csv('/Users/ruben/Documents/MASTER/TFM/data/test_7.csv')
test_7

,Unnamed: 0,ISIN,Name,sector_T,Market,Currency,Ticker,RICS,Tickers_YF,ESG Score,...,totalEsg,environmentScore,socialScore,governanceScore,Company Market Cap,CO2 Equivalent Emissions Total,Market Cap Category,Revenue from Business Activities - Total,carbon intensity,Total CO2 Equivalent Emissions To Revenues USD in million
0,0,SE0015949748,BEIJER REF AB (PUBL),Electrical Components & Equipment,NaN,NaN,0A0H,BEIJb.ST,0A0H.IL,28.635765,...,30.50,8.79,11.02,9.36,8.660336e+10,8795.0,Large Cap,2.263800e+10,0.388506,4.702831
1,1,DK0010181759,CARLSBERG A/S,Brewers,NaN,NaN,0AI4,CARLb.CO,0AI4.IL,75.239436,...,22.56,7.30,8.77,6.49,1.539185e+11,570000.0,Large Cap,7.026500e+10,8.112147,56.352510
2,2,FI0009005961,STORA ENSO OYJ,Paper Products,NaN,NaN,0CXC,STERV.HE,0CXC.IL,89.599284,...,15.93,9.23,2.52,4.18,9.372096e+09,2340000.0,Mid Cap,1.168000e+10,200.342466,202.520252
3,3,PLPEKAO00016,BANK POLSKA KASA OPIEKI SA,Banks,NaN,NaN,0DP0,PEO.WA,0DP0.IL,75.474876,...,26.49,2.07,11.46,12.96,2.522337e+10,72257.4,Large Cap,1.129169e+10,6.399166,33.043888
4,4,PLBZ00000044,BANK ZACHODNI WBK SA,Banks,NaN,NaN,0DVR,SPL1.WA,0DVR.IL,81.673281,...,19.46,0.97,9.06,9.44,3.370204e+10,35132.6,Large Cap,1.216383e+10,2.888285,14.385047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,225,BE0974349814,WDP,Commercial REITs,"Euronext Brussels, Amsterdam",EUR,WDP,WDPP.BR,WDP.DE,57.141401,...,14.88,0.06,7.92,6.90,5.784883e+09,136.0,Mid Cap,2.932500e+08,0.463768,0.429327
226,226,NL0000395903,WOLTERS KLUWER,Professional Information Services,Euronext Amsterdam,EUR,WKL,WLSNc.AS,WKL.AS,68.178890,...,17.21,3.65,9.70,3.85,3.063155e+10,NaN,Large Cap,5.453000e+09,NaN,NaN
227,227,FR0014004EC4,PRECIA,Electrical Components & Equipment,Euronext Paris,EUR,PREC,PREP.PA,WPM,21.552355,...,7.06,0.06,2.65,4.39,1.771509e+08,NaN,Small Cap,NaN,NaN,NaN
228,228,NO0010208051,YARA INTERNATIONAL,Agricultural Chemicals,Oslo Børs,NOK,YAR,YAR.OL,YAR.OL,79.663687,...,26.60,13.81,6.35,6.44,1.175559e+11,17300000.0,Large Cap,2.390200e+10,723.788804,1041.102485


In [17]:
230/2719

0.08458992276572269

In [ ]:
import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

In [ ]:
# Datos de ESG ratings para las compañias TOP Europeas de ESGBook para comparar 

import pandas as pd 

# Data from: https://www.esgbook.com/insights/esg-insights/esg-book-top-100/ -> Selected top ESG rated Companies in EU

df = pd.read_excel('/Users/ruben/Documents/MASTER/TFM/data/esgbook.xlsx', header = None ,names = ['rank', 'company', 'region', 'country', 'sector', 'industry', 'esg', 'temp score', 3, 4, 5])
df_e = df[df['region']== 'Europe']

# Capitalize company names 

df_e['company'] = df_e['company'].str.upper()
df_e.head()

In [ ]:
import pandas as pd
import numpy as np
import math 

path = '/Users/ruben/Documents/MASTER/TFM/data/'
test_5 = pd.read_csv(path + 'test_5.csv')
test_6 = test_5.drop('sector', axis = 1)
test_6 = test_6.iloc[:,[0,3,2,5,6,4,7,1,11,8,9,10,12,13,14,15]] 
#cols = pd.MultiIndex.from_tuples([('basic',"ISIN"), 
#                                  ('basic',"name"), 
#                                  ('basic',"sector_T"),
#                                 ('basic',"market"),
#                                  ('basic','currency'),
#                                  ('TICKERS','Ticker'),
#                                  ('TICKERS','RICS'),
#                                  ('TICKERS','Tickers_YF'),
#                                  ('ESG REF','ESG Score'),
#                                  ('ESG REF','Environmental Pillar Score'),
#                                  ('ESG REF','Social Pillar Score'),
#                                  ('ESG REF','Governance Pillar Score'),
#                                  ('ESG SUS','totalEsg'),
#                                  ('ESG SUS','environmentScore'),
#                                  ('ESG SUS','socialScore'),
#                                  ('ESG SUS','governanceScore')])
#test_6.columns=cols
test_6.head(5) # Queda muy bonico así la verdad

In [ ]:
test_7 = test_6
mask = test_7['ISIN'].str.startswith('CA')
test_7.drop(test_7[mask].index, inplace = True, axis = 0) # Eliminamos las empresas americanas
len(test_7)

In [ ]:
mask = test_7['totalEsg'].isna()
test_7.drop(test_7[mask].index, inplace = True, axis = 0)


In [ ]:
mask = test_7['ESG Score'].isna()
test_7.drop(test_7[mask].index, inplace = True, axis = 0)
len(test_7)

In [ ]:
test_7 = test_7.sort_values(by = 'totalEsg', ascending = False)
test_7

Introducimos los datos necesarios para la métrica: Carbon Intensity, que es la cantidad de CO2 que emite una compañía por cada unidad de producto que vende.

In [ ]:
mkt_cap = ek.get_data(test_7['ISIN'].to_list(), ["TR.CompanyMarketCap(ShType=DEF)"])
mkt_cap[0]['Company Market Cap'].isna().sum()

In [ ]:
emi_co2 = ek.get_data(test_7['ISIN'].to_list(), ["TR.CO2EmissionTotal(Period=FY0)"])
emi_co2 

In [ ]:
test_7 = test_7.merge(mkt_cap[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_7

In [ ]:
test_7 = test_7.merge(emi_co2[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_7

In [ ]:
def classify_market_cap(market_cap):
    if pd.isna(market_cap) or market_cap == '<NA>':
        return 'Unknown'
    elif market_cap < 2000000000:
        return 'Small Cap'
    elif market_cap < 10000000000:
        return 'Mid Cap'
    else:
        return 'Large Cap'
test_7['Market Cap Category'] = test_7['Company Market Cap'].apply(classify_market_cap)
test_7

In [ ]:
test_7.drop(['Instrument_x', 'Instrument_y'], axis = 1, inplace = True)
test_7.head(2)

In [ ]:
rev = ek.get_data(test_7['ISIN'].to_list(), ["TR.F.TotRevenue(Period=FY0)"])
rev 

In [ ]:
test_7 = test_7.merge(rev[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_7.head(2)

In [ ]:
test_7.drop(['Instrument'], axis = 1, inplace = True)

In [ ]:
test_7['carbon intensity'] = test_7['CO2 Equivalent Emissions Total']/(test_7['Revenue from Business Activities - Total']/1000000)
test_7.head()

In [ ]:
rev = ek.get_data(test_7['ISIN'].to_list(), ["TR.AnalyticCO2(Period=FY0)"]) # total cabron emissions is not Scope 1,2&3 hence the difference in CI and last column
test_7 = test_7.merge(rev[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_7.head()

In [ ]:
test_7.drop(['Instrument', 'mktcap_1'], axis = 1, inplace = True)
test_7.head()

In [ ]:
print(test_7.groupby('Market Cap Category')['carbon intensity'].mean(),
    '\n', test_7.groupby('Market Cap Category')['Total CO2 Equivalent Emissions To Revenues USD in million'].mean(),
    '\n', test_7.groupby('Market Cap Category').count()['ISIN'])

# Seguimos, falta MSCI y S&P - Pau quizás?

Nos faltan aun datos de scores pero ya tenemos los datos de carbon intensity, que es la cantidad de CO2 que emite una compañía por cada unidad de producto que vende, para poder usdarlo como weight igual que market cap y clasificacion. 

MKT CAP : 
small cap < 2 Bill  / mid cap 2-10 Bill / large cap > 10 Bill

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the function that will perform the web scraping
def get_esg_score(ticker):
    # Set up the web driver with headless option and wait for page load
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    # Open the MSCI ESG Ratings Climate Search Tool
    driver.get("https://www.msci.com/our-solutions/esg-investing/esg-ratings-climate-search-tool")

    # Accept the cookies banner
    try:
        accept_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Accept All']")))
        accept_button.click()
    except:
        pass

    # Input the ticker and submit the form
    try:
        search_box = wait.until(EC.presence_of_element_located((By.ID, "search")))
        search_box.clear()
        search_box.send_keys(ticker)
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "search-button")))
        search_button.click()
    except:
        driver.quit()
        return pd.DataFrame()

    # Click on the first company that appears in the search results
    try:
        company_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".company-link")))
        company_link.click()
    except:
        driver.quit()
        return pd.DataFrame()

    # Wait for the ESG scores to load and extract them
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ratingdata-container")))
        esg_scores = pd.read_html(driver.page_source, match="ESG RATINGS")[0]
    except:
        driver.quit()
        return pd.DataFrame()

    # Clean up and return the ESG scores
    driver.quit()
    esg_scores.columns = ["Category", "Score"]
    esg_scores.set_index("Category", inplace=True)
    #esg_scores.drop("ESG Score", inplace=True)
    esg_scores.index.name = "ESG Category"
    esg_scores.columns = [ticker]
    return esg_scores


# Test the function with some tickers
tickers = ["AAPL", "GOOG", "AMZN"]
esg_scores = pd.concat([get_esg_score(ticker) for ticker in tickers], axis=1)
print(esg_scores)


In [21]:
import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

import pandas as pd
import numpy as np
import math 

path = '/Users/ruben/Documents/MASTER/TFM/data/'
test_7 = pd.read_csv(path + 'test_7.csv')

In [20]:
# vamos a dividir por sector de manera más amplia, usando la clasificacion de TRBC (Thomson Reuters Business Classification)
# ya que es la que agrupa más compañías y por tanto nos dará una muestra más representativa 
test_7.groupby('sector_T').count()['ISIN']

sector_T
Advertising & Marketing                 2
Agricultural Chemicals                  1
Airport Operators & Services            5
Apparel & Accessories                   4
Apparel & Accessories Retailers         2
                                       ..
Software                                2
Specialty Chemicals                     7
Specialty Mining & Metals               3
Tires & Rubber Products                 1
Wireless Telecommunications Services    2
Name: ISIN, Length: 72, dtype: int64

In [32]:
indus = ek.get_data(test_7['ISIN'].to_list(), ["TR.TRBCIndustryGroup",'TR.TRBCEconomicSector'])
test_8 = test_7.merge(indus[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_8.drop(['Instrument'], axis = 1, inplace = True)
test_8.groupby('TRBC Industry Group Name')[['totalEsg','ESG Score']].mean().sort_values(by = 'totalEsg' ,ascending = True)
test_8.to_csv(path + 'test_8.csv', index = False)

In [52]:
test_8[test_8['TRBC Industry Group Name']=='Leisure Products'][['Name','ESG Score',
       'Environmental Pillar Score', 'Social Pillar Score',
       'Governance Pillar Score', 'totalEsg', 'environmentScore',
       'socialScore', 'governanceScore']] # to find companies in a determined sector

,Name,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,totalEsg,environmentScore,socialScore,governanceScore
145,BENETEAU,50.755799,50.803958,58.474332,40.060837,18.18,1.12,9.88,7.18
217,TRIGANO,41.924203,55.333952,46.260814,31.008315,9.17,0.07,3.15,5.95


In [82]:
for i in range(len(test_8['ISIN'])):
    if test_8['ESG Score'].iloc[i] > test_8['ESG Score'].quantile(0.9) and test_8['totalEsg'].iloc[i] < test_8['totalEsg'].quantile(0.1):
        print(test_8.iloc[i])

Unnamed: 0                                                                              170
ISIN                                                                           FR0010040865
Name                                                                                 GECINA
sector_T                                                                  Diversified REITs
Market                                                                       Euronext Paris
Currency                                                                                EUR
Ticker                                                                                  GFC
RICS                                                                                GFCP.PA
Tickers_YF                                                                           GFC.PA
ESG Score                                                                         88.745818
Environmental Pillar Score                                                      

In [97]:
test_5 = pd.read_csv(path + 'test_5.csv')
indus = ek.get_data(test_5['ISIN'].to_list(), ["TR.TRBCIndustryGroup",'TR.TRBCEconomicSector'])
test_6 = test_5.merge(indus[0], how = 'left', left_on = 'ISIN', right_on = 'Instrument')
test_6.drop(['Instrument'], axis = 1, inplace = True)
test_6.to_csv(path + 'total_comp_clean_industry.csv')

In [279]:
indus_esg_ref = test_6[~test_6['ESG Score'].isna()]
indus_esg_sus = test_6[~test_6['totalEsg'].isna()]

In [611]:
test_6.groupby('TRBC Industry Group Name').count().sort_values(by = 'ISIN', ascending = False)
# Por la falta de compañias en alguna de los sectores, creo conveniente utilizar la media ya que hay demasiadas industrias 
# con pocas compañias y no sería representativo
a = []
for i,k in zip(indus_esg_sus.groupby('TRBC Industry Group Name').count().index, range(len(indus_esg_sus.groupby('TRBC Industry Group Name')))):
    if indus_esg_sus.groupby('TRBC Industry Group Name').count()['ISIN'].iloc[k] < 3:
        a.append(i)
print(len(indus_esg_sus.groupby('TRBC Industry Group Name').count()), len(a), len(a)/len(indus_esg_sus.groupby('TRBC Industry Group Name').count()))
a

#indus_esg_sus.sort_values(by = 'TRBC Industry Group Name', ascending = False)

47 16 0.3404255319148936


['Communications & Networking',
 'Computers, Phones & Household Electronics',
 'Construction Materials',
 'Consumer Goods Conglomerates',
 'Containers & Packaging',
 'Electronic Equipment & Parts',
 'Financial Technology (Fintech) & Infrastructure',
 'Healthcare Providers & Services',
 'Homebuilding & Construction Supplies',
 'Leisure Products',
 'Multiline Utilities',
 'Natural Gas Utilities',
 'Office Equipment',
 'Passenger Transportation Services',
 'Personal & Household Products & Services',
 'Renewable Energy']

In [616]:
indus_esg_sus[indus_esg_sus['TRBC Industry Group Name'].isin(a)].groupby('TRBC Industry Group Name').std()

/var/folders/qz/18ncz94n6j978x_p_8qfmsnh0000gn/T/ipykernel_39796/2661179718.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  indus_esg_sus[indus_esg_sus['TRBC Industry Group Name'].isin(a)].groupby('TRBC Industry Group Name').std()


,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,ESG Score,totalEsg,environmentScore,socialScore,governanceScore
TRBC Industry Group Name,,,,,,,,
Communications & Networking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Computers, Phones & Household Electronics",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Construction Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumer Goods Conglomerates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Containers & Packaging,NaN,NaN,NaN,NaN,10.641957,4.511341,8.492352,6.653875
Electronic Equipment & Parts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Financial Technology (Fintech) & Infrastructure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Healthcare Providers & Services,NaN,NaN,NaN,NaN,3.012275,2.255671,14.163349,5.366940
Homebuilding & Construction Supplies,9.383561,13.718585,0.413972,8.606148,3.061772,1.682914,0.898026,0.480833


In [621]:
means_ref =indus_esg_ref.groupby('TRBC Industry Group Name')[['Environmental Pillar Score',
       'Social Pillar Score', 'Governance Pillar Score', 'ESG Score']].mean().sort_values(by = 'ESG Score' ,ascending = False)
means_ref.to_csv(path + 'means_ref.csv')
means_sus = indus_esg_sus.groupby('TRBC Industry Group Name')[['totalEsg', 'environmentScore', 'socialScore', 'governanceScore']].mean().sort_values(by = 'totalEsg' ,ascending = True)
means_sus.to_csv(path + 'means_sus.csv')

,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,ESG Score
TRBC Industry Group Name,,,,
Communications & Networking,67.842689,88.437030,87.585368,83.537200
Construction Materials,73.260549,77.852241,77.824553,75.911656
Household Goods,71.342890,77.541712,69.434543,73.276867


In [288]:
test_6.to_csv(path + 'total_comp_clean_industry.csv')

In [113]:
def median_diff(value,median):
    return (value-median)/median

# Vemos la distancia relativa del valor a la mediana de la industria para determinar los mejores perfiles de ESG de la industria
# mejor usar los mejores sobre la mediana de la industria que los mejores ESG scores en general? me parece productivo -> Leaders

In [293]:
means_ref['eq weight']  = (means_ref['Environmental Pillar Score'] + means_ref['Social Pillar Score'] + means_ref['Governance Pillar Score'])/3
means_ref['eq weight'].sort_values(ascending = False).head(10)

TRBC Industry Group Name
Communications & Networking                 81.557455
Household Goods                             76.795447
Construction Materials                      76.471343
Metals & Mining                             75.295304
Insurance                                   74.036693
Transport Infrastructure                    73.602960
Textiles & Apparel                          73.540766
Personal & Household Products & Services    73.362428
Beverages                                   73.183554
Multiline Utilities                         73.141523
Name: eq weight, dtype: float64

In [725]:
c =pd.DataFrame(data=test_8['ISIN'], columns = ['ISIN','Environmental Pillar Score', 'Social Pillar Score',
       'Governance Pillar Score', 'ESG Score'])
d =pd.DataFrame(data=test_8['ISIN'], columns = ['ISIN','totalEsg','environmentScore','socialScore','governanceScore'])
b = []
e = []
score_name_ref = ['Environmental Pillar Score',
       'Social Pillar Score', 'Governance Pillar Score', 'ESG Score']
score_name_sus = ['totalEsg', 'environmentScore', 'socialScore', 'governanceScore']
lists = ['1','2','3','4']
k=[]

for i,k in zip(test_8['ISIN'], range(len(test_8['ISIN']))):
    a = test_8[test_8['ISIN']==i]
    if a['TRBC Industry Group Name'].isna().item() == False:
        for v in score_name_ref:
            if means_ref[v][a['TRBC Industry Group Name']].item() > 0 :
                c[v].iloc[k]=median_diff(*a[v],*means_ref[v][a['TRBC Industry Group Name']])
            else:
                c[v].iloc[k]=(a[v])
            b=[]
    else:
        pass


desv_ref = c # Da la diferencia en % a la mediana de la industria de la compañía determinada. 

for i,k in zip(test_8['ISIN'], range(len(test_8['ISIN']))):
    a = test_8[test_8['ISIN']==i]
    if a['TRBC Industry Group Name'].isna().item() == False:
        for v in score_name_sus:
            if means_sus[v][a['TRBC Industry Group Name']].item() > 0 :
                d[v].iloc[k]=median_diff(*a[v],*means_sus[v][a['TRBC Industry Group Name']])
            else:
                d[v].iloc[k]=(a[v])
    else:
        pass
            
desv_sus = d

desv_sus = c.set_index('ISIN')[['totalEsg','environmentScore', 'socialScore', 'governanceScore']]*(-1) # Porque la Formula es X-Mediana/Mediana y queremos que sea Mediana-X/Mediana, ya que SUS es decreciente

desv_ref.to_csv(path + 'desv_ref.csv')
desv_sus.to_csv(path + 'desv_sus.csv')

,ISIN,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,ESG Score
0,SE0015949748,NaN,NaN,NaN,NaN
1,DK0010181759,NaN,NaN,NaN,NaN
2,FI0009005961,NaN,NaN,NaN,NaN
3,PLPEKAO00016,NaN,NaN,NaN,NaN
4,PLBZ00000044,NaN,NaN,NaN,NaN
...,...,...,...,...,...
225,BE0974349814,NaN,NaN,NaN,NaN
226,NL0000395903,NaN,NaN,NaN,NaN
227,FR0014004EC4,NaN,NaN,NaN,NaN
228,NO0010208051,NaN,NaN,NaN,NaN


# Construcción básica del índice

Cogemos las desviaciones y los scores y los cruzamos para obtener una ristra de compañías que son las que vamos a usar para el índice. 

In [ ]:
import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

import pandas as pd
import numpy as np
import math 

path = '/Users/ruben/Documents/MASTER/TFM/data/'
desv_ref = pd.read_csv(path + 'desv_ref.csv')
desv_sus = pd.read_csv(path + 'desv_sus.csv')
test_8 = pd.read_csv(path + 'test_8.csv')
test_6 = pd.read_csv(path + 'total_comp_clean_industry.csv')

In [794]:
# top 50% desviación del score para las compañías dadas 
wp_ref = desv_ref[desv_ref['ESG Score'] > desv_ref['ESG Score'].quantile(0.5)]
wp_sus = desv_sus[desv_sus['totalEsg'] > desv_sus['totalEsg'].quantile(0.5)]
wp_sus.reset_index(inplace = True)
intersec = list(set(wp_sus['ISIN']) & set(wp_ref['ISIN']))
print(intersec.__len__())
mask = desv_ref[desv_ref['ISIN'].isin(intersec)]
opt_1 = test_8[test_8['ISIN'].isin(mask['ISIN'])][['ISIN','Name','ESG Score', 'totalEsg', 'TRBC Industry Group Name']]

65


In [798]:
# top 50% ESG scores para la lista dada 
wp_ref = test_8[test_8['ESG Score'] > test_8['ESG Score'].quantile(0.6)]
wp_sus = test_8[test_8['totalEsg'] < test_8['totalEsg'].quantile(0.6)]
wp_sus.reset_index(inplace = True)
intersec = list(set(wp_sus['ISIN']) & set(wp_ref['ISIN']))
print(intersec.__len__())
mask = test_8[test_8['ISIN'].isin(intersec)]
opt_2 = test_8[test_8['ISIN'].isin(mask['ISIN'])][['ISIN','Name','ESG Score', 'totalEsg', 'TRBC Industry Group Name']]
opt_2

61


,ISIN,Name,ESG Score,totalEsg,TRBC Industry Group Name
2,FI0009005961,STORA ENSO OYJ,89.599284,15.93,Paper & Forest Products
4,PLBZ00000044,BANK ZACHODNI WBK SA,81.673281,19.46,Banking Services
5,ES0130960018,ENAGAS SA,82.503149,14.51,Oil & Gas Related Equipment and Services
10,DE000SYM9999,SYMRISE AG,77.409776,20.32,Chemicals
15,ES0113860A34,BANCO DE SABADELL SA,90.813776,18.82,Banking Services
...,...,...,...,...,...
212,PTSON0AM0001,SONAE,78.563878,17.56,Food & Drug Retailing
216,FR0000051807,TELEPERFORMANCE,80.167645,16.43,Professional & Commercial Services
219,IT0004810054,UNIPOL,80.960697,17.54,Insurance
220,GB00B2B0DG97,RELX,84.661418,18.45,Professional & Commercial Services


In [19]:
65/230

0.2826086956521739

In [801]:
# Top final interseccion de mejor desviación y mejor score -> de los mejores, los que más despuntan en su sector
intersec = list(set(opt_1['ISIN']) & set(opt_2['ISIN']))
print(intersec.__len__())
draft1 = test_8[test_8['ISIN'].isin(intersec)]
draft1.to_csv(path + 'draft1.csv') # Compañías preeliminares del índice compañías

45


In [660]:
inter = pd.read_csv('/Users/ruben/Documents/MASTER/TFM/data/inter.csv')
intersec3 = list(set(intersec) & set(inter['ISIN']))
test_8[test_8['ISIN'].isin(intersec3)].__len__()

,ISIN,mean_ref,mean_sus
0,ES0130960018,0.513035,0.216287
1,SE0000106270,0.214911,0.217422
2,ES0140609019,0.307387,0.209772


In [694]:
total_desv = desv_ref.merge(desv_sus, on = 'ISIN', how = 'inner') # total de las desviaciones agrupados por indústria
tot_desv_ind = total_desv.merge(test_8[['ISIN','TRBC Industry Group Name']], on = 'ISIN', how = 'inner')
tot_desv_ind.columns
index = pd.MultiIndex.from_frame(tot_desv_ind[['TRBC Industry Group Name','ISIN']])
tot_desv_ind.set_index(index, inplace = True)
tot_desv_ind.sort_index(inplace = True)
tot_desv_ind 

Index(['ISIN', 'Environmental Pillar Score', 'Social Pillar Score',
       'Governance Pillar Score', 'ESG Score', 'totalEsg', 'environmentScore',
       'socialScore', 'governanceScore', 'TRBC Industry Group Name'],
      dtype='object')

In [727]:
ce =pd.DataFrame(data=test_8['ISIN'], columns = ['ISIN', 'desv ESG Score', 'desv totalEsg', 'ESG Score', 'totalEsg'])
b = []
c = []

In [783]:
a = 0.6
for i,k in zip(test_8['TRBC Industry Group Name'], range(len(test_8))):

    if tot_desv_ind['ESG Score'].iloc[k] >= tot_desv_ind.loc[i]['ESG Score'].quantile(a):
        ce['desv ESG Score'].iloc[k]=tot_desv_ind['ESG Score'].iloc[k]
        ce['ESG Score'].iloc[k]=test_8['ESG Score'].iloc[k]
    if tot_desv_ind['totalEsg'].iloc[k] >= tot_desv_ind.loc[i]['totalEsg'].quantile(a):
        ce['desv totalEsg'].iloc[k]=tot_desv_ind['totalEsg'].iloc[k]
        ce['totalEsg'].iloc[k]=test_8['totalEsg'].iloc[k]
    else:

        pass
quant_list = ce.dropna() # Compañías seleccionadas por desviación a la indústria del ESG score
print(quant_list.__len__())
quant_list

85


,ISIN,desv ESG Score,desv totalEsg,ESG Score,totalEsg
4,PLBZ00000044,0.074572,0.04249,81.673281,19.46
6,DE0006070006,0.09364,0.248965,90.235372,26.43
7,FI0009005318,0.158982,0.34132,68.480666,12.48
13,SE0001174970,0.207077,0.007624,74.903929,21.69
14,SE0000108847,0.36384,0.180135,24.875136,14.29
...,...,...,...,...,...
212,PTSON0AM0001,0.186253,0.13682,78.563878,17.56
214,IT0004195308,0.206383,0.290399,35.480098,21.21
217,FR0005691656,0.102857,0.107113,41.924203,9.17
221,FR0013326246,0.141663,0.428745,89.76695,4.67


In [784]:
intersec4=quant_list[quant_list['ISIN'].isin(opt_2['ISIN'])]
print(intersec4.__len__())
intersec4
# Lista de los mejores performers por sector cruzado con el top 50% de ESG Score

26


,ISIN,desv ESG Score,desv totalEsg,ESG Score,totalEsg
15,ES0113860A34,0.300461,0.225877,90.813776,18.82
24,SE0000106270,-0.028941,0.082988,76.037371,15.58
34,ES0140609019,0.388001,0.338381,86.593622,17.77
36,DK0060079531,0.173132,0.198413,80.277131,14.08
39,HU0000061726,0.22948,0.060108,76.010146,17.43
41,SE0000112724,0.247748,0.122407,85.346621,18.42
42,SE0016589188,-0.004646,0.331318,85.909357,16.98
53,DK0060227585,0.250562,0.298346,82.529011,15.29
54,PLPZU0000011,0.313247,0.025525,76.610152,18.58
64,AT0000746409,0.225813,0.406524,79.998365,17.83


In [786]:
test_6.ISIN.to_csv('/Users/ruben/Documents/MASTER/TFM/data/ISIN_big.csv', index = False)

# ESG Book
df = pd.read_excel('/Users/ruben/Documents/MASTER/TFM/data/esgbook.xlsx', header = None ,names = ['rank', 'company', 'region', 'country', 'sector', 'industry', 'esg', 'temp score', 3, 4, 5])
df_e = df[df['region']== 'Europe']
# ESG Book tickers
import requests
import time

url = "https://query2.finance.yahoo.com/v1/finance/search"
ticker_fin_1=[]
error = []
retry_count = 5

for i in range(1):
    try:
        for i in df_e['company']: 
            params = {'q': i, 'quotesCount': 1, 'newsCount': 0}
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
            response = requests.get(url, params=params, headers=headers)
            data = response.json()
            if data['quotes']!=[]:
                symbol = data['quotes'][0]['symbol']
                ticker_fin_1.append(symbol)
            else:
                error.append(i)
                ticker_fin_1.append('')
    except IndexError:
        print(f"An error occurred")
        if retry_count > 0:
            print(f"Retrying in 5 seconds... ({retry_count} attempts remaining)")
            time.sleep(5)
            retry_count -= 1
            continue
        else:
            print("Maximum retry attempts reached at Ticker number:", len(ticker_fin_1))
        break

ticker_fin_1